# Overall Workflow

## 1. Data Import and Preprocessing Pipeline

- Import data using appropriate data sources (e.g., CSV, APIs).
- Preprocess the data:
  - Use `pandas` to ensure the format is a proper `Series` or `DataFrame` with a `DatetimeIndex`.
  - Check for and handle missing values, duplicates, or inconsistencies.
  - Verify stock timeline coverage and ensure alignment with market hours or trading periods.
- Run system checks:
  - Confirm that all necessary libraries (e.g., `pandas`, `vectorbt`, `numpy`) are installed.
  - Validate that data is compatible with downstream strategy modules.
- Extract and inspect metadata:
  - Determine stock symbol, exchange, date range, frequency, and other attributes.

## 2. Strategy Development

- Define or select a base strategy model (e.g., moving average crossover, momentum).
- Create a custom strategy using `vectorbt` strategy templates or build from scratch.
- Configure key parameters (e.g., window size, thresholds, position sizing).
- Add optional components like:
  - Position sizing models
  - Risk management rules
  - Leverage constraints or stop-loss mechanisms

## 3. Strategy Testing and Evaluation

- Backtest the strategy using `vectorbt` backtesting modules.
- Analyze performance:
  - Metrics: Sharpe ratio, Calmar ratio, drawdown, win/loss, etc.
  - Visualizations: equity curve, trades, heatmaps
- Compare multiple strategies side-by-side.
- Evaluate tradeoffs, robustness, and risk-reward characteristics.



In [1]:
import vectorbt as vbt
print("vectorbt version:", vbt.__version__)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

vectorbt version: 0.27.3


# Data Pre-Process

In [2]:
from utils.eda_utils import dataset_info_summary, system_info
from data.Data_PipLine import data_from_csv

In [3]:
QQU = r"C:\Users\zhouw\OneDrive\Desktop\Model\data\qqu\QQU ETF Stock Price History.csv"
df = data_from_csv(QQU)
price = df['Close']

# Declare Strategies

In [4]:
import strategy.SimpleStrategy as ss

In [5]:

def compare_strategies(**strategies):
    portfolios = []
    legends = []

    for name, strategy in strategies.items():
        try:
            strategy.run()
            portfolios.append(strategy.portfolio)
            legends.append(name)
        except Exception as e:
            print(f"Error running strategy '{name}': {e}")

    if not portfolios:
        raise ValueError("No valid portfolios to compare.")

    fig = go.Figure()

    for pf, label in zip(portfolios, legends):
            fig.add_trace(go.Scatter(
                x=pf.value().index,
                y=pf.value(),
                mode='lines',
                name=label
            ))

    fig.update_layout(
        title="📈 Strategy Equity Curve Comparison",
        xaxis_title="Date",
        yaxis_title="Portfolio Value",
        template="plotly_dark",
        height=800
    )

    # 📋 Optional: Stats comparison
    stats_df = pd.concat(
        [pf.stats().rename(lambda name: f"{legends[i]}: {name}") for i, pf in enumerate(portfolios)],
        axis=1
    )
    # print(stats_df)

    return fig



def run_plot(**strategies):
    for name, stradegy in  strategies.items():
        try:
            stradegy.run()
            stradegy.plot().show()
        except:
            print(f"Error running strategy '{name}': {e}")
            continue


In [6]:
sma = ss.SMACrossoverStrategy(price)
rsi = ss.RSIStrategy(price)
mom = ss.MomentumStrategy(price)


# Compare and Evaluate Section

In [7]:

run_plot(  
    SMA=sma,
    RSI=rsi,
    Momentum=mom)

compare_strategies(
    SMA=sma,
    RSI=rsi,
    Momentum=mom
).show()

c:\Users\zhouw\OneDrive\Desktop\Model\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\zhouw\OneDrive\Desktop\Model\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\zhouw\OneDrive\Desktop\Model\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\zhouw\OneDrive\Desktop\Model\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

c:\Users\zhouw\OneDrive\Desktop\Model\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\zhouw\OneDrive\Desktop\Model\venv\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\U